<a href="https://colab.research.google.com/github/yorkjong/vistock/blob/feature%2Fmansfield/mansfield.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Stock Charts Inspired by Stan Weinstein

This notebook features stock charts inspired by Stan Weinstein's book, *Secrets for Profiting in Bull and Bear Markets*.

### Install and Setup (this section will be executed automatically)

#### Install Required Packages

In [1]:
%pip install "git+https://github.com/yorkjong/vistock.git@feature/mansfield"
%pip install requests-cache

!wget -O NotoSansTC-Regular.ttf https://share.cole.tw/d/Tools%20-%20MAC/Fonts/Noto_Sans_TC/static/NotoSansTC-Regular.ttf?sign=bATsZP5QZdI_2EM15sAbcAE48Cacle91CpwUNOCMuM8=:0

  Cloning https://github.com/yorkjong/vistock.git (to revision feature/mansfield) to /tmp/pip-req-build-i35tv1au
  Running command git clone --filter=blob:none --quiet https://github.com/yorkjong/vistock.git /tmp/pip-req-build-i35tv1au
  Running command git checkout -b feature/mansfield --track origin/feature/mansfield
  Switched to a new branch 'feature/mansfield'
  Branch 'feature/mansfield' set up to track remote branch 'feature/mansfield' from 'origin'.
  Resolved https://github.com/yorkjong/vistock.git to commit f5696febcdf29793109cb70a745a6c60102c24ab
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 2.4 MB/s eta 0:00:00
  Created wheel for vistock: filename=vistock-0.5.0-py3-none-any.whl size=76882 sha256=9fb26cac05e396c1a5e7a3743aa95f3b0f3b3650f9f27f2a710c238f7681265e
  Stored in directory: /tmp/pip-ephem-wheel-cache-cdwjxzzn/wheels/e1/e8/cb/23b949814f97377684075b0c16826efc8969661b3ddd4469f2
Successfully built vistock
   ━━━━━━━━━

In [2]:
# @title Enable Requests Cache
import requests_cache
requests_cache.install_cache('ibd_cache', expire_after=3600)

#### Setup and Configuration

In [3]:
# @title Set Chinese Font for matplotlib
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt

fm.fontManager.addfont('NotoSansTC-Regular.ttf')
font_name = 'Noto Sans TC'
if font_name not in plt.rcParams['font.sans-serif']:
    plt.rcParams['font.sans-serif'].insert(0, font_name)

In [4]:
# @title Dropdown Menus
import ipywidgets as widgets

def create_style_dropdown(desc=None, value=None):
    return widgets.Dropdown(
        options=['default', 'classic', 'yahoo', 'charles', 'tradingview', 'binance', 'binancedark', 'mike', 'nightclouds', 'checkers', 'ibd', 'sas', 'starsandstripes', 'kenan', 'blueskies', 'brasil'],
        value='yahoo' if value is None else value,
        description='Style:' if desc is None else desc,
        style={'description_width': 'initial'},
    )

def create_template_dropdown(desc=None, value=None):
    return widgets.Dropdown(
        options=['plotly', 'plotly_white', 'plotly_dark', 'ggplot2', 'seaborn', 'simple_white', 'presentation', 'xgridoff', 'ygridoff'],
        value='plotly_dark' if value is None else value,
        description='Template:' if desc is None else desc,
        style={'description_width': 'initial'},
    )

In [5]:
# @title Multiple Searchable Dropdown Menus
import ipywidgets as widgets

def create_search_box():
    '''Create a Text widget for search input'''
    return widgets.Text(
        description='Search:',
        placeholder='Type to search',
        layout=widgets.Layout(width='auto')
    )

def create_dropdown(options, description='Stock:'):
    '''Create a Dropdown widget for displaying filtered options'''
    return widgets.Dropdown(
        description=description,
        options=[None] + options,  # None as the default option
        layout=widgets.Layout(width='auto'),
        value=None  # Set default value to None
    )

def update_dropdown(change, dropdown, options):
    '''Update the options in the dropdown based on search input'''
    search_text = change['new'].lower()
    filtered = [option for option in options if search_text in option.lower()]
    if filtered:
        dropdown.options = [None] + filtered
        dropdown.value = filtered[0]  # Auto-select the first matching option
    else:
        dropdown.options = [None]  # Retain only the None option if no match

def remove_duplicates_preserve_order(lst):
    '''Remove duplicates from a list while preserving order'''
    seen = set()
    result = []
    for item in lst:
        if item and item not in seen:
            seen.add(item)
            result.append(item)
    return result

def get_dropdowns_selected_options(dropdowns):
    '''Get selected options from the dropdowns'''
    selected = [dropdown.value for dropdown in dropdowns if dropdown.value]
    return remove_duplicates_preserve_order(selected)

def create_search_dropdowns(options, max_selections):
    '''Create a layout with search boxes and dropdowns'''
    # Create UI components
    search_boxes = [create_search_box() for _ in range(max_selections)]
    dropdowns = [create_dropdown(options) for _ in range(max_selections)]

    # Bind search box and dropdown menu events
    for search_box, dropdown in zip(search_boxes, dropdowns):
        search_box.observe(lambda change, dropdown=dropdown:
                            update_dropdown(change, dropdown, options),
                            names='value')
    # Create the layout
    controls = [widgets.HBox([search_box, dropdown])
                for search_box, dropdown  in zip(search_boxes, dropdowns)]
    layout = widgets.VBox(controls)
    return dropdowns, layout

In [6]:
# @title enable_plotly_in_cell
# ref. https://stackoverflow.com/questions/76593068/plotly-figure-not-rendering-in-ipywidgets-interact-function-google-colab
def enable_plotly_in_cell():
  import IPython
  from plotly.offline import init_notebook_mode
  display(IPython.core.display.HTML('''<script src="/static/components/requirejs/require.js"></script>'''))
  init_notebook_mode(connected=False)

### Glossary of Terms

#### Source (The Source of Stocks to Analyze)
- **Description**: Stocks can be sourced from various exchanges or indices.
- **Common Abbreviations**:
  - **Taiwan Markets**:
    - `TWSE`: Taiwan Stock Exchange (台灣上市股票交易所)
    - `TPEX`: Taipei Exchange (上櫃交易所)
    - `ESB`: Emerging Stock Board (興櫃交易所)
    - Combine with `+` (e.g., `TWSE+TPEX`, `TWSE+TPEX+ESB`)
  - **America Markets**:
    - `SPX`: S&P 500 (標普五百指數)
    - `DJIA`: Dow Jones Industrial Average (道瓊指數)
    - `NDX`: NASDAQ-100 (納斯達克一百指數)
    - `SOX`: PHLX Semiconductor Index (費半指數)
    - Combine with `+` (e.g., `SPX+DJIA+NDX+SOX`)

#### Period (Historical Data Time Range)
- **Description**: The duration for fetching historical data.
- **Example**: `2y` for 2 years

#### Interval (Historical Data Frequency)
- **Description**: The frequency of historical data points.
- **Common Intervals**:
  - `'1d'`: Daily data
  - `'1wk'`: Weekly data
  - `'1mo'`: Monthly data

#### MA (Moving Average)
- **SMA**: Simple Moving Average
- **EMA**: Exponential Moving Average

#### RS (Relative Strength)
- **Description**: Measures a stock's performance relative to a benchmark index.
- **Unit**: Percentage (%)
- **Interpretation**:
  - A value of 0 represents the performance of the benchmark index or market.
  - A positive value indicates outperformance relative to the benchmark.
  - A negative value indicates underperformance relative to the benchmark.

##### References:
- [Mansfield Relative Strength (Original Version) by stageanalysis — Indicator by Stage_Analysis — TradingView](https://www.tradingview.com/script/NzUBDDtb-Mansfield-Relative-Strength-Original-Version-by-stageanalysis/)
- [Mansfield Relative Strength | TrendSpider Store](https://trendspider.com/trading-tools-store/indicators/mansfield-relative-strength/)
- [Mansfield Relative Strength | ChartMill.com](https://www.chartmill.com/documentation/technical-analysis/indicators/35-Mansfield-Relative-Strength)
- [How to create the Mansfield Relative Performance Indicator - Stage Analysis](https://www.stageanalysis.net/blog/4266/how-to-create-the-mansfield-relative-performance-indicator)
  - [Stan Weinstein's Stage Analysis | Page 49 | Trade2Win Forums • UK Financial Trading Community](https://www.trade2win.com/threads/stan-weinsteins-stage-analysis.134944/page-49#post-2137398)


### Plots

In [7]:
# @title _ {"run":"auto"}
source = "Taiwan Stocks" #@param ["U.S. Stocks", "Taiwan Stocks"]
period = "2y" # @param ["6mo", "1y","2y","5y"]
interval = "1wk" # @param ["1d","1wk"]
backend = "mplfinance" # @param ["mplfinance","Plotly"]

import ipywidgets as widgets
from IPython.display import display, clear_output
import vistock.stock_indices as si
import vistock.tw as tw
import vistock.mpl.mansfield as mpl_mansfield
import vistock.plotly.mansfield as ply_mansfield

MAX_SELECTIONS = 8

src_codes = {
    'U.S. Stocks': 'SPX+DJIA+NDX+RUI+SOX',
    'Taiwan Stocks': 'TWSE+TPEX+ESB'
}[source]
symbols = si.get_tickers(src_codes)
if source == 'Taiwan Stocks':
    symbols += [tw.stock_name(s) for s in symbols]
dropdowns, layout = create_search_dropdowns(symbols, MAX_SELECTIONS)

stock_theme_dropdown = {
    'mplfinance': create_style_dropdown('Stock Theme:', 'yahoo'),
    'Plotly': create_template_dropdown('Stock Theme:', 'simple_white'),
}[backend]
cmp_theme_dropdown = {
    'mplfinance': create_style_dropdown('Comparison Theme:', 'checkers'),
    'Plotly': create_template_dropdown('Comparison Theme:', 'plotly_dark'),
}[backend]
theme_dropdowns = widgets.VBox([stock_theme_dropdown, cmp_theme_dropdown])

btn_plot = widgets.Button(description="Plot a Chart")
btn_clear_one = widgets.Button(description="Clear One")
btn_clear_all = widgets.Button(description="Clear All")
buttons = widgets.HBox([btn_plot, btn_clear_one, btn_clear_all])
outputs = widgets.VBox()

display(layout, theme_dropdowns, buttons, outputs)

rs_cmp = {
    'mplfinance': mpl_mansfield.RelativeStrengthLines,
    'Plotly': ply_mansfield.RelativeStrengthLines,
}[backend]

stock_chart = {
    'mplfinance': mpl_mansfield.StockChart,
    'Plotly': ply_mansfield.StockChart,
}[backend]

# Button click event
def on_plot_click(b):
    selected = get_dropdowns_selected_options(dropdowns)
    if source == 'Taiwan Stocks':
        selected = [tw.stock_name(s) for s in selected]
        selected = remove_duplicates_preserve_order(selected)
    new_output = widgets.Output()
    with new_output:
        if not selected:
            print("No stocks selected. Please select up to 8 stocks.")
        else:
            chart = rs_cmp if len(selected) > 1 else stock_chart
            if len(selected) == 1:
                selected = selected[0]
                theme = stock_theme_dropdown.value
            else:
                theme = cmp_theme_dropdown.value
            if backend == 'mplfinance':
                chart.plot(selected, interval=interval, period=period,
                           style=theme)
            else: # Plotly
                chart.plot(selected, interval=interval, period=period,
                           template=theme)
    outputs.children = (new_output,) + outputs.children

def on_clear_one_click(b):
    if outputs.children:
        children = list(outputs.children)
        children.pop(0)
        outputs.children = tuple(children)

def on_clear_all_click(b):
    outputs.children = ()

btn_plot.on_click(on_plot_click)
btn_clear_one.on_click(on_clear_one_click)
btn_clear_all.on_click(on_clear_all_click)

if backend == 'Plotly':
    enable_plotly_in_cell()


VBox()